In [306]:
import numpy as np
import pandas as pd
import yfinance as yf

In [78]:
def pct(df, periods=0):
    shifted = df['Close'].shift(periods=periods)
    return (df['Close'] - shifted) / shifted

def history(*, ticker, start, end):
    ticker = yf.Ticker(ticker)
    history = ticker.history(start=start, end=end)
    history['1day'] = pct(history, 1)
    history['1mo'] = pct(history, 20)
    splits = history['Stock Splits']
    split_dates = splits[splits != 0]
    if not split_dates.empty:
        dates = ', '.join(str(d) for d in split_dates.keys())
        print("Splits found on ticker %s: %s" % (ticker, dates))
    return history

In [79]:
start, end = '2019-12-01', '2021-02-17'
tickers = 'ARKF SQ VOO VTI QQQ XLK NIO AAPL ARKG TSLA'.split()
data = {t: history(ticker=t, start=start, end=end) for t in tickers}

Splits found on ticker yfinance.Ticker object <AAPL>: 2020-08-31 00:00:00
Splits found on ticker yfinance.Ticker object <TSLA>: 2020-08-31 00:00:00


In [81]:
class Brain:
    def __init__(self, data):
        self.data = data

    def _extract_pct(self, ticker):
        self.data[ticker][['1day', '1mo']].columns = []

,Open,High,Low,Close,Volume,Dividends,Stock Splits,1day,1mo
Date,,,,,,,,,
2019-12-02,68.949997,68.959999,66.110001,66.879997,6758200,0,0,NaN,NaN
2019-12-03,65.300003,67.330002,64.680000,67.040001,4432700,0,0,0.002392,NaN
2019-12-04,67.470001,68.430000,67.190002,67.779999,3715100,0,0,0.011038,NaN
2019-12-05,67.589996,67.889999,66.660004,67.139999,4265000,0,0,-0.009442,NaN
2019-12-06,67.669998,68.322998,67.610001,67.980003,4058400,0,0,0.012511,NaN
...,...,...,...,...,...,...,...,...,...
2021-02-09,256.579987,262.570007,250.550003,258.119995,10815100,0,0,-0.007116,0.144656
2021-02-10,259.769989,263.149994,252.750000,257.489990,8644600,0,0,-0.002441,0.131725
2021-02-11,262.000000,269.665009,258.500000,265.929993,10191800,0,0,0.032778,0.151611


In [101]:
def join_column(data, column):
    def series(ticker):
        df = data[ticker][column]
        df.name = ticker
        return df

    return pd.concat([series(t) for t in data.keys()], axis=1)

In [326]:
buy_date, sell_date = '2020-01-01', '2020-12-31'
sell_date_plus_1 = str(np.datetime64(sell_date) + np.timedelta64(1, 'D'))

df_day = join_column(data, '1day').loc[buy_date:sell_date_plus_1]
df_month = join_column(data, '1mo').loc[buy_date:sell_date_plus_1]
df_close = join_column(data, 'Close').loc[buy_date:sell_date_plus_1]

sell_price = df_close.loc[sell_date]
ticker_count = 1 / df_close[df_close.notnull()].count(axis=1)

In [327]:
allocated = df_close.copy()
allocated[:] = daily_investment
allocated = allocated.mul(ticker_count, axis=0)
shares = allocated / df_close
sell_value = shares.sum() * sell_price
total_value = sell_value.sum()
total_invested = allocated.sum().sum()
yld = (total_value - total_invested) / total_invested
print("Total yield, equal allocation strategy:", round(yld * 100, 1), '%')

Total yield, equal allocation strategy: 133.8 %


In [328]:
achievement = df_day.sub(df_day.max(axis=1), axis=0)
allocated = achievement.div(achievement.sum(axis=1), axis=0)
shares = allocated / df_close
sell_value = shares.sum() * sell_price
total_value = sell_value.sum()
total_invested = allocated.sum().sum()
yld = (total_value - total_invested) / total_invested
print("Total yield, biggest loser strategy:", round(yld * 100, 1), '%')

Total yield, biggest loser strategy: 153.2 %


In [329]:
n_filter = 2

worst = df_month.apply(lambda s: s.nsmallest(n_filter).index.tolist(), axis=1)

def modify(s):
    rv = s.copy()
    rv[worst[s.name]] = s.max()
    return rv

updated = df_day.apply(modify, axis=1)
achievement = updated.sub(updated.max(axis=1), axis=0)
allocated = achievement.div(achievement.sum(axis=1), axis=0)
shares = allocated / df_close
sell_value = shares.sum() * sell_price
total_value = sell_value.sum()
total_invested = allocated.sum().sum()
yld = (total_value - total_invested) / total_invested
print("Total yield, filtered(", n_filter, ") biggest loser strategy:", round(yld * 100, 1), '%')

Total yield, filtered( 2 ) biggest loser strategy: 145.9 %


In [330]:
winner_buffer = 1.4
achievement = df_day.sub(df_day.max(axis=1) * winner_buffer, axis=0)
allocated = achievement.div(achievement.sum(axis=1), axis=0)
shares = allocated / df_close
sell_value = shares.sum() * sell_price
total_value = sell_value.sum()
total_invested = allocated.sum().sum()
yld = (total_value - total_invested) / total_invested
print("Total yield, biggest loser w/winner(", winner_buffer, ") strategy:", round(yld * 100, 1), '%')

Total yield, biggest loser w/winner( 1.4 ) strategy: 251.3 %
